# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 08**: Apache Spark + Kafka Integration

**Fecha**: 4 de abril del 2025

**Nombre del Estudiante**: Marco Albanese, Vicente Siloe

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://f8e470126b03:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1eefb5cf-1da0-4c1a-b058-c560a668f52f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

### Creación del Kafka Stream

In [4]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "9bc654f7b2fc:9093") \
                .option("subscribe", "lab08-equipo-mcqueen") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform binary data into string

In [5]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

### Configuración del "Sink" del stream

In [6]:

query = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(10)

25/04/04 13:57:34 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-54ea4aba-aab6-49fd-bab4-a96e22492a99. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/04 13:57:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/04 13:57:35 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-----+---------+------+---------+-------------+---------+
|key|value|topic|partition|offset|timestamp|timestampType|value_str|
+---+-----+-----+---------+------+---------+-------------+---------+
+---+-----+-----+---------+------+---------+-------------+---------+



False

-------------------------------------------
Batch: 1
-------------------------------------------
+----+-------------------------------------------------------------+--------------------+---------+------+-----------------------+-------------+--------------------+
|key |value                                                        |topic               |partition|offset|timestamp              |timestampType|value_str           |
+----+-------------------------------------------------------------+--------------------+---------+------+-----------------------+-------------+--------------------+
|NULL|[6C 61 62 30 38 20 65 71 75 69 70 6F 20 6D 63 71 75 65 65 6E]|lab08-equipo-mcqueen|0        |0     |2025-04-04 13:57:51.136|0            |lab08 equipo mcqueen|
+----+-------------------------------------------------------------+--------------------+---------+------+-----------------------+-------------+--------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+----------------------------------------------------------------------+--------------------+---------+------+-----------------------+-------------+-----------------------+
|key |value                                                                 |topic               |partition|offset|timestamp              |timestampType|value_str              |
+----+----------------------------------------------------------------------+--------------------+---------+------+-----------------------+-------------+-----------------------+
|NULL|[76 69 65 72 6E 65 73 20 34 20 64 65 20 61 62 72 69 6C 20 32 30 32 35]|lab08-equipo-mcqueen|0        |1     |2025-04-04 14:01:03.182|0            |viernes 4 de abril 2025|
+----+----------------------------------------------------------------------+--------------------+---------+------+-----------------------+-------------+-----------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+----+-------------------------------------------------------------------------------------------+--------------------+---------+------+-----------------------+-------------+------------------------------+
|key |value                                                                                      |topic               |partition|offset|timestamp              |timestampType|value_str                     |
+----+-------------------------------------------------------------------------------------------+--------------------+---------+------+-----------------------+-------------+------------------------------+
|NULL|[70 72 6F 63 65 73 61 6D 69 65 6E 74 6F 20 64 65 20 64 61 74 6F 73 20 6D 61 73 69 76 6F 73]|lab08-equipo-mcqueen|0        |2     |2025-04-04 14:01:08.876|0            |procesamiento de datos masivos|
+----+-------------------------------------------------------------------------

-------------------------------------------
Batch: 4
-------------------------------------------
+----+----------------------------------------------------------------------------------------+--------------------+---------+------+-----------------------+-------------+-----------------------------+
|key |value                                                                                   |topic               |partition|offset|timestamp              |timestampType|value_str                    |
+----+----------------------------------------------------------------------------------------+--------------------+---------+------+-----------------------+-------------+-----------------------------+
|NULL|[6D 61 72 63 6F 20 61 6C 62 61 6E 65 73 65 2C 20 76 69 63 65 6E 74 65 20 73 69 6C 6F 65]|lab08-equipo-mcqueen|0        |3     |2025-04-04 14:01:37.386|0            |marco albanese, vicente siloe|
+----+----------------------------------------------------------------------------------------+

In [7]:
query.stop()

In [8]:
sc.stop()